### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
5729,2023-11-07,Brasil Nbb,19:00,Cerrado,União Corinthians,2.57,1.49,150.5,1.87,1.87,3.5,2.00,1.72
5730,2023-11-07,Brasil Nbb,20:00,Flamengo,Botafogo,1.01,13.50,158.5,1.83,1.83,-21.5,2.00,1.72
5731,2023-11-07,Brasil Nbb,20:00,Franca,Corinthians Paulista,1.21,4.34,160.5,1.86,1.88,-11.5,2.05,1.69
5732,2023-11-07,Europa Eurocopa,14:00,Lietkabelis,JL Bourg,2.23,1.64,160.5,1.88,1.86,1.5,2.10,1.65
5733,2023-11-07,Europa Eurocopa,16:00,Venezia,Cedevita Olimpija,1.25,3.84,160.5,1.89,1.87,-9.5,2.00,1.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5793,2023-11-07,Rússia Superliga,12:00,Dome-Springs Izhevsk,M. Khimki,7.75,1.06,155.5,1.83,1.83,8.5,1.95,1.73
5794,2023-11-07,Rússia Superliga,13:00,CSKA Moscow 2,Tambov,1.98,1.74,150.5,1.83,1.83,1.5,1.83,1.83
5795,2023-11-07,Rússia Superliga,13:00,MBA Moscow 2,University-Ugra,1.68,2.06,152.5,1.83,1.83,-3.5,2.04,1.66
5796,2023-11-07,Suécia Superettan,15:00,KFUM Fryshuset,AIK Basket,1.74,1.98,163.5,1.80,1.86,-3.5,1.91,1.76


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
30,22:00,Eua Ncaa,North Texas,Northern Iowa,130.5,1.91,0.9980,Over
33,23:00,Eua Ncaa,Depaul,IPFW,143.5,1.91,0.8765,Over
